# Semantic Search - OHS Legislation (Turkish)


### Import the Needed Packages

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
import sys

In [4]:
sys.path.append('../../../../DLAIUtils')

In [5]:
#!pip install python-dotenv
#!pip install datasets==2.15.0
#!pip install PyPDF2
#!pip install openai
#!pip install google-generativeai
!pip install -U google-generativeai

In [6]:
from datasets import load_dataset
from sentence_transformers import SentenceTransformer
from pinecone import Pinecone, ServerlessSpec
from DLAIUtils import Utils
import os
import time
import torch
from dotenv import load_dotenv
import PyPDF2
from openai import OpenAI
import google.generativeai as genai

# Load environment variables from .env file
load_dotenv()

True

In [7]:
from tqdm.auto import tqdm

### Load the OHS Legislation PDF Dataset

In [8]:
# Load ISG Mevzuat PDF
pdf_path = 'data/isg_mevzuat.pdf'

def extract_text_from_pdf(pdf_path):
    """Extract text from PDF file"""
    text = ""
    with open(pdf_path, 'rb') as file:
        pdf_reader = PyPDF2.PdfReader(file)
        for page in pdf_reader.pages:
            text += page.extract_text() + "\n"
    return text

# Extract text from PDF
pdf_text = extract_text_from_pdf(pdf_path)
print(f"Total characters extracted: {len(pdf_text)}")
print(f"First 500 characters:\n{pdf_text[:500]}")

Total characters extracted: 234007
First 500 characters:
Bu doküman Global Belgelendirme Hizmetleri A.Ş. tarafından 01.06.2021 tarihinde oluşturulmuştur.  
  
 
 
 
 
 
 
 
 
4857 SAYILI İŞ KANUNU  
6331 SAYILI İŞ SAĞLIĞI VE GÜVENLİĞİ 
KANUNU  
BAĞLI DİĞER MEVZUAT  
 
 
 
 
 
 
 
 
 
 
 
1 
 İÇİNDEKİLER  
4857 SAYILI İŞ KANUNU  ...........................................................................................................  4 
Alt İşverenlik Yönetmeliği  .......................................................................................


In [9]:
# Improved text chunking to preserve complete sentences
def split_text_into_chunks(text, chunk_size=800, overlap=200):
    """Split text into overlapping chunks at sentence boundaries"""
    import re
    
    # Split into sentences (Turkish-aware)
    sentence_endings = r'[.!?]\s+'
    sentences = re.split(sentence_endings, text)
    
    chunks = []
    current_chunk = ""
    
    for sentence in sentences:
        sentence = sentence.strip()
        if not sentence:
            continue
            
        # If adding this sentence exceeds chunk_size and we have content, save chunk
        if len(current_chunk) + len(sentence) > chunk_size and current_chunk:
            chunks.append(current_chunk.strip())
            # Keep overlap: start new chunk with last part of previous chunk
            words = current_chunk.split()
            overlap_words = words[-int(len(words) * 0.2):] if len(words) > 5 else []
            current_chunk = ' '.join(overlap_words) + ' ' + sentence
        else:
            current_chunk += ' ' + sentence
    
    # Add the last chunk
    if current_chunk.strip():
        chunks.append(current_chunk.strip())
    
    return chunks

# Create chunks from PDF text with better sentence preservation
text_chunks = split_text_into_chunks(pdf_text, chunk_size=800, overlap=200)
print(f"Number of text chunks: {len(text_chunks)}")
print(f"\nFirst 3 chunks:")
for i, chunk in enumerate(text_chunks[:3]):
    print(f"\n--- Chunk {i+1} (length: {len(chunk)}) ---")
    print(chunk[:300] + "..." if len(chunk) > 300 else chunk)

Number of text chunks: 405

First 3 chunks:

--- Chunk 1 (length: 699) ---
Bu doküman Global Belgelendirme Hizmetleri A.Ş tarafından 01.06.2021 tarihinde oluşturulmuştur 4857 SAYILI İŞ KANUNU  
6331 SAYILI İŞ SAĞLIĞI VE GÜVENLİĞİ 
KANUNU  
BAĞLI DİĞER MEVZUAT  
 
 
 
 
 
 
 
 
 
 
 
1 
 İÇİNDEKİLER  
4857 SAYILI İŞ KANUNU  .....................................................

--- Chunk 2 (length: 779) ---
11 Çocuk ve Genç İşçilerin Çalıştırılma Usul ve Esasları Hakkında Yönetmelik ............................ 12 
Gebe veya Emziren Kadınların Çalıştırılma Şartlarıyla İlgili Yönetmelik  .................................. 13 
Haftalık İş Günlerine Bölünemeyen Çalışma Süreleri Yönetmeliği  .................

--- Chunk 3 (length: 751) ---
Çalışma Yönetmeliği .............................. 17 Kadın Ça lışanların Gece Postalarında Çalıştırılma Koşulları Hakkında Yönetmelik .................. 18 
Postalar Halinde İşçi Çalıştırılarak Yürütülen İşlerde Çalışmalara İlişkin Yönetmelik  ..........

In [10]:
# Clean and prepare ISG mevzuat chunks
def clean_text(text):
    # Replace problematic characters with ASCII equivalents
    text = text.replace('–', '-')  # en dash
    text = text.replace('—', '-')  # em dash
    text = text.replace(''', "'")  # left single quotation mark
    text = text.replace(''', "'")  # right single quotation mark
    text = text.replace('"', '"')  # left double quotation mark
    text = text.replace('"', '"')  # right double quotation mark
    # Keep Turkish characters but remove other problematic Unicode
    # Don't encode to ASCII for Turkish text
    return text

# Clean the chunks
cleaned_chunks = [clean_text(chunk) for chunk in text_chunks if len(chunk.strip()) > 50]

print(f'Number of cleaned chunks: {len(cleaned_chunks)}')
print('\nFirst 2 chunks:')
for i, chunk in enumerate(cleaned_chunks[:2]):
    print(f"\n--- Chunk {i+1} ---")
    print(chunk[:300] + "..." if len(chunk) > 300 else chunk)

Number of cleaned chunks: 405

First 2 chunks:

--- Chunk 1 ---
Bu doküman Global Belgelendirme Hizmetleri A.Ş tarafından 01.06.2021 tarihinde oluşturulmuştur 4857 SAYILI İŞ KANUNU  
6331 SAYILI İŞ SAĞLIĞI VE GÜVENLİĞİ 
KANUNU  
BAĞLI DİĞER MEVZUAT  
 
 
 
 
 
 
 
 
 
 
 
1 
 İÇİNDEKİLER  
4857 SAYILI İŞ KANUNU  .....................................................

--- Chunk 2 ---
11 Çocuk ve Genç İşçilerin Çalıştırılma Usul ve Esasları Hakkında Yönetmelik ............................ 12 
Gebe veya Emziren Kadınların Çalıştırılma Şartlarıyla İlgili Yönetmelik  .................................. 13 
Haftalık İş Günlerine Bölünemeyen Çalışma Süreleri Yönetmeliği  .................


### Check cuda and Setup the model

**Note**: "Checking cuda" refers to checking if you have access to GPUs (faster compute). In this course, we are using CPUs. So, you might notice some code cells taking a little longer to run.

We are using *all-MiniLM-L6-v2* sentence-transformers model that maps sentences to a 384 dimensional dense vector space.

In [11]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
if device != 'cuda':
    print('Sorry no cuda.')
model = SentenceTransformer('all-MiniLM-L6-v2', device=device)

Sorry no cuda.


In [12]:
query = 'işçi sağlığı ve güvenliği nedir?'
xq = model.encode(query)
xq.shape

(384,)

In [13]:
utils = Utils()

### Setup Pinecone

In [14]:
# Load API keys from .env file
PINECONE_API_KEY = os.getenv('PINECONE_API_KEY')
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
GOOGLE_API_KEY = os.getenv('GOOGLE_API_KEY')

# Verify Pinecone API key is loaded
if not PINECONE_API_KEY or PINECONE_API_KEY == 'your_pinecone_api_key_here':
    raise ValueError("Please set PINECONE_API_KEY in .env file")

# Configure Google Gemini
if GOOGLE_API_KEY:
    genai.configure(api_key=GOOGLE_API_KEY)
    print("✓ API keys loaded from .env file (OpenAI + Google Gemini)")
else:
    print("✓ API keys loaded from .env file (OpenAI only)")

✓ API keys loaded from .env file (OpenAI + Google Gemini)


In [15]:
pinecone = Pinecone(api_key=PINECONE_API_KEY)
INDEX_NAME = utils.create_dlai_index_name('isg')

# Check and delete existing index if present
if INDEX_NAME in [index.name for index in pinecone.list_indexes()]:
    print(f"Deleting existing index: {INDEX_NAME}")
    pinecone.delete_index(INDEX_NAME)
    time.sleep(5)  # Wait for deletion to complete

print(f"Creating new index: {INDEX_NAME}")
pinecone.create_index(
    name=INDEX_NAME, 
    dimension=model.get_sentence_embedding_dimension(), 
    metric='cosine',
    spec=ServerlessSpec(cloud='aws', region='us-east-1')
)

# Wait for index to be ready
print("Waiting for index to be ready...")
time.sleep(10)

# Connect to index
index = pinecone.Index(INDEX_NAME)
print(f"✓ Index ready: {index}")
print(f"✓ Index stats: {index.describe_index_stats()}")

Deleting existing index: isg-86hbriq-qif5lhm6p-ygourxdueztik7oaqa
Creating new index: isg-86hbriq-qif5lhm6p-ygourxdueztik7oaqa
Creating new index: isg-86hbriq-qif5lhm6p-ygourxdueztik7oaqa
Waiting for index to be ready...
Waiting for index to be ready...
✓ Index ready: <pinecone.db_data.index.Index object at 0x3070be350>
✓ Index ready: <pinecone.db_data.index.Index object at 0x3070be350>
✓ Index stats: {'_response_info': {'raw_headers': {'connection': 'keep-alive',
                                    'content-length': '150',
                                    'content-type': 'application/json',
                                    'date': 'Wed, 14 Jan 2026 18:47:39 GMT',
                                    'grpc-status': '0',
                                    'server': 'envoy',
                                    'x-envoy-upstream-service-time': '43',
                                    'x-pinecone-request-id': '5770634670681135803',
                                    'x-pinecone-req

### Create Embeddings and Upsert to Pinecone

In [16]:
batch_size = 100
# Use all chunks from ISG mevzuat
isg_chunks = cleaned_chunks

# Create a mapping to store the actual text (for later retrieval)
text_mapping = {i: text for i, text in enumerate(isg_chunks)}

print(f"Total chunks to process: {len(isg_chunks)}")

for i in tqdm(range(0, len(isg_chunks), batch_size)):
    # find end of batch
    i_end = min(i + batch_size, len(isg_chunks))
    # create IDs batch
    ids = [f"isg_{x}" for x in range(i, i_end)]
    # create embeddings
    xc = model.encode(isg_chunks[i:i_end])
    # create metadata batch - store only chunk_id to avoid encoding issues
    metadatas = [{'chunk_id': idx} for idx in range(i, i_end)]
    # create records list for upsert with proper format
    records = [
        {
            'id': id_,
            'values': values.tolist(),
            'metadata': metadata
        }
        for id_, values, metadata in zip(ids, xc, metadatas)
    ]
    # upsert to Pinecone
    index.upsert(vectors=records)

print(f"\n✓ Successfully uploaded {len(isg_chunks)} ISG mevzuat chunks to Pinecone")
print(f"Text mapping created with {len(text_mapping)} entries")

Total chunks to process: 405


  0%|          | 0/5 [00:00<?, ?it/s]


✓ Successfully uploaded 405 ISG mevzuat chunks to Pinecone
Text mapping created with 405 entries


In [17]:
index.describe_index_stats()

{'_response_info': {'raw_headers': {'connection': 'keep-alive',
                                    'content-length': '185',
                                    'content-type': 'application/json',
                                    'date': 'Wed, 14 Jan 2026 18:47:49 GMT',
                                    'grpc-status': '0',
                                    'server': 'envoy',
                                    'x-envoy-upstream-service-time': '4',
                                    'x-pinecone-request-id': '5054425798342982642',
                                    'x-pinecone-request-latency-ms': '3',
                                    'x-pinecone-response-duration-ms': '5'}},
 'dimension': 384,
 'index_fullness': 0.0,
 'memoryFullness': 0.0,
 'metric': 'cosine',
 'namespaces': {'__default__': {'vector_count': 405}},
 'storageFullness': 0.0,
 'total_vector_count': 405,
 'vector_type': 'dense'}

### Run Your Queries - Multi-AI Provider Support

In [ ]:
# Initialize OpenAI client
openai_client = OpenAI(api_key=OPENAI_API_KEY)

# Enhanced query function with multiple AI providers and response length control
def run_query(query, use_ai=True, ai_provider="openai", use_reasoning=False, max_length="normal", top_k=5):
    """
    Query ISG legislation with semantic search and optional AI enhancement
    
    Args:
        query: The question to ask (in Turkish)
        use_ai: Whether to use AI for response synthesis (default: True)
        ai_provider: "openai", "gemini", or "none" (default: "openai")
        use_reasoning: Use advanced reasoning models (default: False)
        max_length: "short" (3-4 sentences), "medium" (5-7 sentences), or "normal" (detailed)
        top_k: Number of relevant chunks to retrieve (default: 5)
    """
    embedding = model.encode(query).tolist()
    results = index.query(top_k=top_k, vector=embedding, include_metadata=True, include_values=False)
    
    if not use_ai or ai_provider == "none":
        # Simple mode: just print the chunks
        print("🔍 Most Relevant Results:\n")
        for i, result in enumerate(results['matches'], 1):
            chunk_id = result['metadata']['chunk_id']
            text = text_mapping.get(chunk_id, "Text not found")
            print(f"{i}. [Score: {round(result['score'], 2)}]\n{text}\n")
    else:
        # AI mode: use AI to synthesize a better answer
        if ai_provider == "gemini":
            if use_reasoning:
                print("🧠 Deep Analysis with Gemini 2.5 Pro:\n")
            else:
                print("🤖 Enhanced Response with Gemini 2.5 Flash:\n")
        else:  # openai
            if use_reasoning:
                print("🧠 Deep Analysis with GPT-4 (o1 Reasoning):\n")
            else:
                print("🤖 Enhanced Response with GPT-4 mini:\n")
        
        # Gather relevant chunks
        relevant_texts = []
        for result in results['matches']:
            chunk_id = result['metadata']['chunk_id']
            text = text_mapping.get(chunk_id, "")
            if text:
                relevant_texts.append(text)
        
        context = "\n\n".join(relevant_texts)
        
        # Length-aware instructions
        if max_length == "short":
            length_instruction = "VERY IMPORTANT: Limit your response to maximum 3-4 sentences. Only mention the most important points."
            max_tokens = 2048
        elif max_length == "medium":
            length_instruction = "Keep your response concise (5-7 sentences)."
            max_tokens = 3072
        else:  # normal or long
            length_instruction = "Provide a detailed and complete response."
            max_tokens = 4096
        
        system_prompt = f"""You are an expert on OHS (Occupational Health and Safety) legislation. 
Answer questions based ONLY on the provided legislation texts. 
Provide answers in proper Turkish, in a bullet-point format, and reference relevant articles.
{length_instruction}"""

        user_prompt = f"""Answer the question based on the following OHS legislation texts.

Question: {query}

Legislation Texts:
{context}

Please:
1. Answer the question fully and clearly
2. Use ONLY the information from the provided legislation texts
3. Provide the answer in proper Turkish, in bullet-point format
4. Reference relevant articles
5. Give practical examples
6. Highlight important points
{length_instruction}"""

        # Call AI provider
        if ai_provider == "gemini":
            # Use Gemini 1.5 Flash for both normal and reasoning (FREE!)
            # Note: Using same model for both to avoid quota limits
            gemini_model = genai.GenerativeModel('models/gemini-2.5-flash')
            response = gemini_model.generate_content(
                f"{system_prompt}\n\n{user_prompt}",
                generation_config={
                    'temperature': 0.3,
                    'max_output_tokens': max_tokens,
                    'top_p': 0.95,
                    'top_k': 40
                }
            )
            if use_reasoning:
                print("💭 Used Gemini 2.5 Flash (with reasoning mode)\n")
            else:
                print("🤖 Enhanced Response with Gemini 2.5 Flash:\n")
            print(response.text)
        else:  # openai
            if use_reasoning:
                # Use o1 model for deep reasoning
                response = openai_client.chat.completions.create(
                    model="o1-preview",
                    messages=[
                        {"role": "user", "content": f"{system_prompt}\n\n{user_prompt}"}
                    ]
                )
                print("💭 Used O1 Reasoning Model (with internal reasoning)\n")
                print(response.choices[0].message.content)
            else:
                # Use GPT-4 for standard response
                response = openai_client.chat.completions.create(
                    model="gpt-4o-mini",
                    messages=[
                        {"role": "system", "content": system_prompt},
                        {"role": "user", "content": user_prompt}
                    ],
                    temperature=0.3,
                    max_tokens=max_tokens
                )
                print(response.choices[0].message.content)
        
        print("\n" + "="*80)
        print(f"📚 Source Texts (showing top {min(3, len(relevant_texts))}):")
        for i, text in enumerate(relevant_texts[:3], 1):
            print(f"\n{i}. {text[:200]}...")
    
    return results

In [ ]:
# GPT-4o-mini - Normal length response
run_query('işçi sağlığı ve güvenliği ile ilgili temel kurallar nelerdir?', 
          use_ai=True, ai_provider="openai", max_length="normal")

🤖 Enhanced Response with GPT-4 Turbo:

İşçi sağlığı ve güvenliği ile ilgili temel kurallar aşağıda belirtilmiştir:

- **Koruyucu ve Önleyici Hizmetler**: İşveren, işyerindeki sağlık ve güvenlik risklerini önlemek için koruyucu hizmetleri yürütmek üzere bir veya birden fazla kişiyi görevlendirmek zorundadır. (İş Sağlığı ve Güvenliği Yönetmeliği)

- **Görevli Kişilerin Hakları**: Sağlık ve güvenlikle görevli kişiler, bu görevleri nedeniyle dezavantajlı duruma düşmemeli ve yeterli zaman almalıdır. (İş Sağlığı ve Güvenliği Yönetmeliği)

- **Çalışanların Bilgilendirilmesi**: Çalışanlar ve temsilcileri, iş sağlığı ve güvenliği hizmetlerinin amaç ve usulleri hakkında bilgilendirilmelidir. (İş Sağlığı ve Güvenliği Yönetmeliği)

- **Uygun Talimatlara Uyum**: Çalışanlar, iş sağlığı ve güvenliği ile ilgili mevzuata uygun talimatlara uymalıdır. (İş Sağlığı ve Güvenliği Yönetmeliği)

- **İş Birliği**: İş sağlığı ve güvenliği hizmetlerini yerine getirmek üzere işveren tarafından görevlendirilen kişi

QueryResponse(matches=[{'id': 'isg_14',
 'metadata': {'chunk_id': 14},
 'score': 0.697583258,
 'values': []}, {'id': 'isg_10',
 'metadata': {'chunk_id': 10},
 'score': 0.695350707,
 'values': []}, {'id': 'isg_376',
 'metadata': {'chunk_id': 376},
 'score': 0.694004118,
 'values': []}, {'id': 'isg_212',
 'metadata': {'chunk_id': 212},
 'score': 0.687288821,
 'values': []}, {'id': 'isg_231',
 'metadata': {'chunk_id': 231},
 'score': 0.68340975,
 'values': []}], namespace='', usage={'read_units': 1}, _response_info={'raw_headers': {'date': 'Wed, 14 Jan 2026 19:42:02 GMT', 'content-type': 'application/json', 'content-length': '445', 'connection': 'keep-alive', 'x-pinecone-max-indexed-lsn': '5', 'x-pinecone-request-latency-ms': '35', 'x-pinecone-request-id': '8336903275851413552', 'x-envoy-upstream-service-time': '36', 'x-pinecone-response-duration-ms': '37', 'grpc-status': '0', 'server': 'envoy'}})

In [ ]:
# Gemini 2.5 Flash - Deep reasoning (FREE!)
query = 'iş yerinde güvenlik önlemleri nelerdir?'
run_query(query, use_ai=True, ai_provider="gemini", use_reasoning=True, max_length="normal")

🧠 Deep Analysis with Gemini 1.5 Pro:

💭 Used Gemini 1.5 Flash (with reasoning mode)

Sağlanan mevzuat metinlerine göre iş yerinde alınması gereken güvenlik önlemleri şunlardır:

*   **Mesleki Eğitimler:** Tehlikeli ve çok tehlik

📚 Source Texts (showing top 3):

1. Tam süreli işyeri hekimi görevlendirilen işyerlerinde, diğer sağlık personeli görevlendirilmesi zorunlu değildir İş sağlığı ve güvenliği hizmetle rinin desteklenmesi : 
İş sağlığı ve güvenliği hizmetl...

2. 76 Tehlikeli ve Çok Tehlikeli Sınıfta Yer Alan İşlerde Çalıştırılacakların Mesleki Eğitimler i Hk Yön 78 
Tozla Mücadele Yönetmeliği  ........................................ ............................

3. Önlemleri Hakkında Yönetmelik ..... 69 Kimyasal Maddelerle Çalışmalarda Sağlık ve Güvenlik Önlemleri Hakkında Yönetmelik ........ 69 
Kişisel Koruyucu Donanım Yönetmeliği  ...............................


QueryResponse(matches=[{'id': 'isg_107',
 'metadata': {'chunk_id': 107},
 'score': 0.54851824,
 'values': []}, {'id': 'isg_10',
 'metadata': {'chunk_id': 10},
 'score': 0.548188806,
 'values': []}, {'id': 'isg_9', 'metadata': {'chunk_id': 9}, 'score': 0.546551824, 'values': []}, {'id': 'isg_216',
 'metadata': {'chunk_id': 216},
 'score': 0.540629447,
 'values': []}, {'id': 'isg_393',
 'metadata': {'chunk_id': 393},
 'score': 0.529871523,
 'values': []}], namespace='', usage={'read_units': 1}, _response_info={'raw_headers': {'date': 'Wed, 14 Jan 2026 19:03:39 GMT', 'content-type': 'application/json', 'content-length': '443', 'connection': 'keep-alive', 'x-pinecone-max-indexed-lsn': '5', 'x-pinecone-request-latency-ms': '6', 'x-pinecone-request-id': '8932299798722278594', 'x-envoy-upstream-service-time': '6', 'x-pinecone-response-duration-ms': '8', 'grpc-status': '0', 'server': 'envoy'}})

In [37]:
query = 'What is the capital of France?'
run_query(query)

🤖 Enhanced Response with GPT-4 Turbo:

Verilen yasal metinler, Fransa'nın başkenti ile ilgili herhangi bir bilgi içermediğinden, bu soruya doğrudan bir yanıt verilemez. Ancak, sağlanan metinler iş sağlığı ve güvenliği (İSG) ile ilgili yönetmeliklere odaklanmaktadır. Bu nedenle, Fransa'nın başkenti hakkındaki sorunun yanıtı bu metinlerde bulunamaz. 

Yasal metinlerin içeriği özetle:
- Çalışanların patlayıcı ortamların tehlikelerinden korunması
- Titreşimle ilgili risklerden korunma
- Ekranlı araçlarla çalışmada sağlık ve güvenlik önlemleri
- Elle taşıma işleri
- Geçici veya belirli süreli işlerde iş sağlığı ve güvenliği
- İş ekipmanlarının kullanımında sağlık ve güvenlik şartları
- Gece çalışma süreleri ve koşulları
- Telafi çalışması
- Postalar halinde işçi çalıştırılmasıyla ilgili düzenlemeler
- Kadın çalışanların gece postalarında çalıştırılma koşulları
- Uzaktan çalışma, ücret ve prim ödemeleri, yıllık ücretli izin gibi konulara dair yönetmelikler

Bu metinler, iş sağlığı ve güvenli

QueryResponse(matches=[{'id': 'isg_5', 'metadata': {'chunk_id': 5}, 'score': 0.159964085, 'values': []}, {'id': 'isg_80',
 'metadata': {'chunk_id': 80},
 'score': 0.156729698,
 'values': []}, {'id': 'isg_39',
 'metadata': {'chunk_id': 39},
 'score': 0.151911736,
 'values': []}, {'id': 'isg_78',
 'metadata': {'chunk_id': 78},
 'score': 0.150190353,
 'values': []}, {'id': 'isg_2', 'metadata': {'chunk_id': 2}, 'score': 0.148008347, 'values': []}], namespace='', usage={'read_units': 1}, _response_info={'raw_headers': {'date': 'Wed, 14 Jan 2026 19:04:05 GMT', 'content-type': 'application/json', 'content-length': '436', 'connection': 'keep-alive', 'x-pinecone-max-indexed-lsn': '5', 'x-pinecone-request-latency-ms': '6', 'x-pinecone-request-id': '2226510731035491807', 'x-envoy-upstream-service-time': '6', 'x-pinecone-response-duration-ms': '7', 'grpc-status': '0', 'server': 'envoy'}})

In [ ]:
# GPT-4o-mini - Medium response (5-7 sentences)
query = 'çalışanların eğitim yükümlülükleri nelerdir?'
run_query(query, use_ai=True, ai_provider="openai", max_length="medium")

🤖 Enhanced Response with GPT-4 Turbo:

- Çalışanların eğitim yükümlülükleri, iş sağlığı ve güvenliği açısından ilgili mevzuatın öngördüğü esaslara uymak ve işverenin verdiği talimatlara riayet etmekle başlar. (Çalışanların yükümlülükleri)
- İşe alınmadan önce, yeterlilik ekindeki (Ek-1) çizelgede yer alan işlerde çalışacakların mesleki eğitime tabi tutulmaları zorunludur. (Mesleki eğitim zorunluluğu)
- Çalışanlar, işyerindeki makine, cihaz ve tehlikeli maddeleri kurallara uygun şekilde kullanmakla yükümlüdürler. (Çalışanların yükümlülükleri)
- Güvenlik donanımlarını doğru kullanmak, keyfi olarak çıkarmamak ve değiştirmemek de eğitim yükümlülükleri arasındadır. (Çalışanların yükümlülükleri)
- Ayrıca, acil durumlar hakkında düzenli tatbikatlar yapılması ve ilk yardım donanımının sağlanması gerekmektedir. (Acil Durumlar Hakkında Yönetmelik)
- Örneğin, bir çalışan tehlikeli bir madde ile çalışıyorsa, bu maddeye ilişkin güvenlik bilgi formlarını bilmesi ve uygun koruyucu ekipman kullanması 

QueryResponse(matches=[{'id': 'isg_311',
 'metadata': {'chunk_id': 311},
 'score': 0.731275499,
 'values': []}, {'id': 'isg_13',
 'metadata': {'chunk_id': 13},
 'score': 0.679422438,
 'values': []}, {'id': 'isg_298',
 'metadata': {'chunk_id': 298},
 'score': 0.675013602,
 'values': []}, {'id': 'isg_128',
 'metadata': {'chunk_id': 128},
 'score': 0.670695364,
 'values': []}, {'id': 'isg_10',
 'metadata': {'chunk_id': 10},
 'score': 0.668191969,
 'values': []}], namespace='', usage={'read_units': 1}, _response_info={'raw_headers': {'date': 'Wed, 14 Jan 2026 19:07:57 GMT', 'content-type': 'application/json', 'content-length': '446', 'connection': 'keep-alive', 'x-pinecone-max-indexed-lsn': '5', 'x-pinecone-request-latency-ms': '7', 'x-pinecone-request-id': '5816653808191648580', 'x-envoy-upstream-service-time': '6', 'x-pinecone-response-duration-ms': '8', 'grpc-status': '0', 'server': 'envoy'}})


### Side-by-Side Comparison: OpenAI vs Gemini vs No AI

**3 AI Providers:**
1. **No AI** (`use_ai=False`): Raw chunks only - instant results
2. **OpenAI GPT-4o-mini** (`ai_provider="openai"`): High quality, paid, fast
3. **Google Gemini** (`ai_provider="gemini"`): Fast, FREE with generous limits! 


In [45]:
query1 = "İşverenin iş sağlığı ve güvenliği konusundaki genel yükümlülükleri maddeler halinde nelerdir?"
run_query(query1, use_ai=True, ai_provider="openai", max_length="long")

🤖 Enhanced Response with GPT-4 Turbo:

İşverenin iş sağlığı ve güvenliği konusundaki genel yükümlülükleri aşağıdaki gibidir:

- **Çalışanların Bilgilendirilmesi**: İşveren, iş sağlığı ve güvenliği ile ilgili işaretlerin anlamlarını ve bu işaretlerin gerektirdiği davranış biçimlerini çalışanlara açıklamalıdır. (Referans: Belirtilen yönetmelik)

- **Eğitim Sağlanması**: İşveren, çalışanların iş sağlığı ve güvenliği konularında eğitim almasını sağlamakla yükümlüdür. Bu eğitim, çalışanların sosyal, kültürel ve ekonomik yeteneklerinin gelişimini destekleyecek şekilde planlanmalıdır. (Referans: Belirtilen yönetmelik)

- **Çalışanların Görüşlerinin Alınması**: İşveren, iş sağlığı ve güvenliği ile ilgili konularda çalışanların veya temsilcilerinin görüşlerini almalı ve katılımlarını sağlamalıdır. (Referans: Belirtilen yönetmelik)

- **Mesleki Eğitim Zorunluluğu**: İşveren, yeterlilik ekindeki çizelgede yer alan işlerde çalışacak olanların işe alınmadan önce mesleki eğitime tabi tutulmalarını s

QueryResponse(matches=[{'id': 'isg_274',
 'metadata': {'chunk_id': 274},
 'score': 0.714277327,
 'values': []}, {'id': 'isg_308',
 'metadata': {'chunk_id': 308},
 'score': 0.684712887,
 'values': []}, {'id': 'isg_311',
 'metadata': {'chunk_id': 311},
 'score': 0.671235,
 'values': []}, {'id': 'isg_386',
 'metadata': {'chunk_id': 386},
 'score': 0.668987274,
 'values': []}, {'id': 'isg_376',
 'metadata': {'chunk_id': 376},
 'score': 0.668402672,
 'values': []}], namespace='', usage={'read_units': 1}, _response_info={'raw_headers': {'date': 'Wed, 14 Jan 2026 19:47:37 GMT', 'content-type': 'application/json', 'content-length': '447', 'connection': 'keep-alive', 'x-pinecone-max-indexed-lsn': '5', 'x-pinecone-request-latency-ms': '83', 'x-pinecone-request-id': '707509671234199645', 'x-envoy-upstream-service-time': '77', 'x-pinecone-response-duration-ms': '84', 'grpc-status': '0', 'server': 'envoy'}})

In [46]:
query2 = "Gebe veya emziren kadın çalışanların gece postalarında çalıştırılması ile ilgili yasal düzenlemeler nelerdir?"
run_query(query2, use_ai=True, ai_provider="openai", max_length="normal")

🤖 Enhanced Response with GPT-4 Turbo:

- **Gece Postası Tanımı**: Gece postası, İş Kanunu’nda belirtilen gece çalışma sürelerini kapsayan ve yedi buçuk saati geçmeyen çalışma zamanını ifade eder. (Kaynak: Gece Çalıştırma Koşulları Hakkında Yönetmelik)

- **Çalışma Süresi**: Kadın çalışanlar, gece postalarında yedi buçuk saatten fazla çalıştırılamaz. (Kaynak: Gece Çalıştırma Koşulları Hakkında Yönetmelik)

- **Sağlık Raporu**: Kadın çalışanların gece postalarında çalıştırılabilmesi için, işe başlamadan önce işyeri hekiminden, gece postalarında çalıştırılmalarında sakınca olmadığına dair sağlık raporu alınması gerekmektedir. (Kaynak: Gece Çalıştırma Koşulları Hakkında Yönetmelik)

- **Sağlık Gözetimi**: İşveren, çalışanın özel durumunu ve işyerinde maruz kalınan sağlık ve güvenlik risklerini dikkate alarak, işyeri hekimince belirlenen düzenli aralıklarla sağlık muayenelerinin yapılmasını sağlamalıdır. (Kaynak: Gece Çalıştırma Koşulları Hakkında Yönetmelik)

- **Ulaşım Yükümlülüğü**: İşve

QueryResponse(matches=[{'id': 'isg_78',
 'metadata': {'chunk_id': 78},
 'score': 0.751935065,
 'values': []}, {'id': 'isg_73',
 'metadata': {'chunk_id': 73},
 'score': 0.745189667,
 'values': []}, {'id': 'isg_75',
 'metadata': {'chunk_id': 75},
 'score': 0.742697775,
 'values': []}, {'id': 'isg_81',
 'metadata': {'chunk_id': 81},
 'score': 0.734445572,
 'values': []}, {'id': 'isg_76',
 'metadata': {'chunk_id': 76},
 'score': 0.721873343,
 'values': []}], namespace='', usage={'read_units': 1}, _response_info={'raw_headers': {'date': 'Wed, 14 Jan 2026 19:48:27 GMT', 'content-type': 'application/json', 'content-length': '440', 'connection': 'keep-alive', 'x-pinecone-max-indexed-lsn': '5', 'x-pinecone-request-latency-ms': '38', 'x-pinecone-request-id': '432019448568884317', 'x-envoy-upstream-service-time': '39', 'x-pinecone-response-duration-ms': '40', 'grpc-status': '0', 'server': 'envoy'}})

In [47]:
query3 = "Bir iş yerinde risk değerlendirmesi yapılırken hangi adımlar izlenmeli ve kimler bu ekibe dahil edilmelidir?"
run_query(query3, use_ai=True, ai_provider="openai", max_length="long")

🤖 Enhanced Response with GPT-4 Turbo:

- **Risk Değerlendirmesi Adımları:**
  - Tehlikelerin belirlenmesi: İşyerindeki tehlikeler ayrı ayrı tespit edilir ve bu tehlikelerin kaynaklanabileceği riskler analiz edilir. (Madde 60)
  - Maruziyetin belirlenmesi: Çalışanların maruz kaldığı gürültü düzeyi gibi faktörler değerlendirilir ve gerektiğinde ölçümler yapılır. (Madde 60)
  - Mevcut kontrol tedbirlerinin etkisinin göz önünde bulundurulması: Alınan önlemlerin etkinliği değerlendirilir. (Madde 60)
  - Risklerin azaltılması için uygun tedbirlerin alınması: Risk değerlendirmesi sonucunda ortaya çıkan riskler için önlemler belirlenir. (Madde 60)

- **Risk Değerlendirmesi Ekibi:**
  - İşveren veya işveren vekili
  - İşyerinde sağlık ve güvenlik hizmetini yürüten iş güvenliği uzmanları ve işyeri hekimleri
  - İşyerindeki çalışan temsilcileri
  - İşyerindeki destek elemanları
  - İşyerindeki tüm birimleri temsil edecek şekilde belirlenen, mevcut veya muhtemel tehlike kaynakları ile riskler konu

QueryResponse(matches=[{'id': 'isg_239',
 'metadata': {'chunk_id': 239},
 'score': 0.783207893,
 'values': []}, {'id': 'isg_152',
 'metadata': {'chunk_id': 152},
 'score': 0.768857,
 'values': []}, {'id': 'isg_248',
 'metadata': {'chunk_id': 248},
 'score': 0.733007431,
 'values': []}, {'id': 'isg_143',
 'metadata': {'chunk_id': 143},
 'score': 0.719381332,
 'values': []}, {'id': 'isg_230',
 'metadata': {'chunk_id': 230},
 'score': 0.71781826,
 'values': []}], namespace='', usage={'read_units': 1}, _response_info={'raw_headers': {'date': 'Wed, 14 Jan 2026 19:52:18 GMT', 'content-type': 'application/json', 'content-length': '446', 'connection': 'keep-alive', 'x-pinecone-max-indexed-lsn': '5', 'x-pinecone-request-latency-ms': '44', 'x-pinecone-request-id': '7303910797970160024', 'x-envoy-upstream-service-time': '44', 'x-pinecone-response-duration-ms': '46', 'grpc-status': '0', 'server': 'envoy'}})

In [48]:
query4 = "İş kazası ve meslek hastalığı arasındaki fark nedir? Yasal olarak nasıl tanımlanmışlardır?"
run_query(query4, use_ai=True, ai_provider="openai", max_length="normal")

🤖 Enhanced Response with GPT-4 Turbo:

- **İş Kazası Tanımı**: İş kazası, çalışanın işyerinde veya işin yürütülmesi sırasında, işin gereğiyle ilgili bir olay sonucu meydana gelen, çalışanın bedensel veya ruhsal sağlığında bir bozulmaya yol açan durumdur. Ancak, sağlanan metinlerde iş kazası ile ilgili spesifik bir tanım verilmemiştir.

- **Meslek Hastalığı Tanımı**: Meslek hastalığı, çalışanın yaptığı işin niteliği nedeniyle maruz kaldığı risk faktörleri sonucu ortaya çıkan, belirli bir hastalığın iş ile bağlantılı olarak gelişmesidir. Yine, sağlanan metinlerde meslek hastalığı ile ilgili spesifik bir tanım verilmemiştir.

- **Farklar**:
  - **Zaman ve Mekan**: İş kazası, işin yürütülmesi sırasında aniden meydana gelirken; meslek hastalığı, uzun süreli maruziyet sonucunda gelişir.
  - **Oluşum Şekli**: İş kazası genellikle bir olay sonucu (örneğin, düşme, yaralanma) oluşurken; meslek hastalığı, belirli bir süre boyunca maruz kalınan risk etmenleri (örneğin, kimyasal maddeler, gürültü) 

QueryResponse(matches=[{'id': 'isg_164',
 'metadata': {'chunk_id': 164},
 'score': 0.701391757,
 'values': []}, {'id': 'isg_47',
 'metadata': {'chunk_id': 47},
 'score': 0.668949187,
 'values': []}, {'id': 'isg_217',
 'metadata': {'chunk_id': 217},
 'score': 0.6684075,
 'values': []}, {'id': 'isg_311',
 'metadata': {'chunk_id': 311},
 'score': 0.660653114,
 'values': []}, {'id': 'isg_88',
 'metadata': {'chunk_id': 88},
 'score': 0.660349429,
 'values': []}], namespace='', usage={'read_units': 1}, _response_info={'raw_headers': {'date': 'Wed, 14 Jan 2026 19:54:24 GMT', 'content-type': 'application/json', 'content-length': '444', 'connection': 'keep-alive', 'x-pinecone-max-indexed-lsn': '5', 'x-pinecone-request-latency-ms': '37', 'x-pinecone-request-id': '7914920284965318894', 'x-envoy-upstream-service-time': '37', 'x-pinecone-response-duration-ms': '38', 'grpc-status': '0', 'server': 'envoy'}})

In [49]:
query5 = "İş yerlerinde acil durum planı hazırlanırken hangi hususlar dikkate alınmalı ve tatbikatlar ne sıklıkla yapılmalıdır?"
run_query(query5, use_ai=True, ai_provider="openai", max_length="normal")

🤖 Enhanced Response with GPT-4 Turbo:

- **Acil Durum Planı Hazırlarken Dikkate Alınması Gereken Hususlar:**
  - Çalışma ortamında bulunan fiziksel, kimyasal, biyolojik, psikososyal, ergonomik ve benzeri tehlike kaynakları belirlenmeli ve kayda alınmalıdır. (Referans: 59. madde)
  - İşveren, çalışma ortamı, kullanılan maddeler ve iş ekipmanı ile çevre şartlarını dikkate alarak muhtemel acil durumları önceden değerlendirmelidir. (Referans: İşverenin yükümlülükleri)
  - Acil durumların olumsuz etkilerini önleyici ve sınırlandırıcı tedbirler alınmalıdır. (Referans: İşverenin yükümlülükleri)
  - Acil durum planları hazırlanırken, benzer işyerlerinde meydana gelen iş kazaları ve meslek hastalıkları da değerlendirilebilir. (Referans: 59. madde)

- **Tatbikatların Yapılması:**
  - Acil durum planlarının tatbikatlarının düzenli olarak yapılması gerekmektedir. Ancak, spesifik bir sıklık belirtilmemiştir. İşveren, acil durum planlarının etkinliğini sağlamak için tatbikatları düzenli aralıklarla 

QueryResponse(matches=[{'id': 'isg_233',
 'metadata': {'chunk_id': 233},
 'score': 0.697142601,
 'values': []}, {'id': 'isg_374',
 'metadata': {'chunk_id': 374},
 'score': 0.690549791,
 'values': []}, {'id': 'isg_223',
 'metadata': {'chunk_id': 223},
 'score': 0.679239213,
 'values': []}, {'id': 'isg_213',
 'metadata': {'chunk_id': 213},
 'score': 0.675946176,
 'values': []}, {'id': 'isg_258',
 'metadata': {'chunk_id': 258},
 'score': 0.675868034,
 'values': []}], namespace='', usage={'read_units': 1}, _response_info={'raw_headers': {'date': 'Wed, 14 Jan 2026 19:55:39 GMT', 'content-type': 'application/json', 'content-length': '450', 'connection': 'keep-alive', 'x-pinecone-max-indexed-lsn': '5', 'x-pinecone-request-latency-ms': '77', 'x-pinecone-request-id': '2875606279567787563', 'x-envoy-upstream-service-time': '77', 'x-pinecone-response-duration-ms': '79', 'grpc-status': '0', 'server': 'envoy'}})

# Gemini 

In [ ]:
# Complex question with Gemini 2.5 Flash Reasoning (FREE!)
run_query(
    'işveren ve çalışanların iş kazalarını önlemedeki sorumlulukları nelerdir ve bunlar nasıl denetlenir?',
    use_ai=True,
    ai_provider="gemini",
    use_reasoning=True,
    max_length="normal"
)

🧠 Deep Analysis with Gemini 1.5 Pro:

💭 Used Gemini 1.5 Flash (with reasoning mode)

Sağlanan mevzuat metinlerine göre, işveren ve çalışanların iş kazalarını önlemedeki sorumlulukları ve denetim mekanizmaları aşağıdaki gibidir:

**

📚 Source Texts (showing top 3):

1. bilgilendirir İşveren işaretlerin anlamları ve bu işare tlerin gerektirdiği davranış biçimleri hakkında, çalışanların eğitim almasını sağlar Çalışanların görüşlerinin alınması ve katılımlarının sağlan...

2. yer alan işyerlerinde, iş sağlığı ve güvenliği hizmetlerinin işveren veya işveren vekili tarafından yürütülebilmesine ilişkin usul ve esasları belirlemektir Tanımlar:  
İşveren : Bu Yönetmelik kapsamı...

3. tedbirler hakkında, 2) İşyerinde görevlendirilen kişiler hakkında, işçilerin ve temsilcilerinin gerekli bilgiyi almalarını sağlamak zorundadır b) İşveren, başka işyerlerinden çalışmak üzere kendi işye...


QueryResponse(matches=[{'id': 'isg_308',
 'metadata': {'chunk_id': 308},
 'score': 0.716405,
 'values': []}, {'id': 'isg_274',
 'metadata': {'chunk_id': 274},
 'score': 0.711962759,
 'values': []}, {'id': 'isg_386',
 'metadata': {'chunk_id': 386},
 'score': 0.705380499,
 'values': []}, {'id': 'isg_217',
 'metadata': {'chunk_id': 217},
 'score': 0.695505202,
 'values': []}, {'id': 'isg_102',
 'metadata': {'chunk_id': 102},
 'score': 0.689708769,
 'values': []}], namespace='', usage={'read_units': 1}, _response_info={'raw_headers': {'date': 'Wed, 14 Jan 2026 19:40:53 GMT', 'content-type': 'application/json', 'content-length': '447', 'connection': 'keep-alive', 'x-pinecone-max-indexed-lsn': '5', 'x-pinecone-request-latency-ms': '71', 'x-pinecone-request-id': '6722410950312404594', 'x-envoy-upstream-service-time': '72', 'x-pinecone-response-duration-ms': '73', 'grpc-status': '0', 'server': 'envoy'}})

In [52]:
query_gemini1 = "Tehlikeli ve çok tehlikeli sınıfta yer alan işyerleri için iş güvenliği uzmanı ve işyeri hekimi görevlendirme sürelerini karşılaştırarak detaylandır."
run_query(query_gemini1, use_ai=True, ai_provider="gemini", use_reasoning=True, max_length="long")

🧠 Deep Analysis with Gemini 1.5 Pro:

💭 Used Gemini 1.5 Flash (with reasoning mode)

Sağlanan mevzuat metinleri incelendiğinde, tehlikeli ve çok tehlikeli sınıfta yer alan işyerleri için iş güvenliği uzmanı ve işyeri hekimi görevlendirme sürelerine ilişkin doğrudan bir bilgi bulunmamaktadır.

Mevzuat metinlerinde yer alan ilgili bilgiler şunlardır:

*   **İş Güvenliği Uzmanı ve İşyeri Hekimi İstihdamı Zorunluluğu:**
    *   Mevzuat, "iş sözleşmesiyle çalışan en az bir işyeri hekimi, iş güvenliği uzmanı, diğer sağlık personeli, istihdamı zorunludur" ifadesiyle bu profesyonellerin istihdam edilmesinin zorunlu olduğunu belirtmektedir. Ancak bu ifade, görevlendirme sürelerine (örneğin, ayda kaç saat veya çalışan başına düşen süre gibi) dair bir detay içermemektedir.
*   **İş Sağlığı ve Güvenliği Hizmetlerinin Desteklenmesi:**
    *   "İŞ SAĞLIĞI VE GÜVENLİĞİ HİZMETLERİNİN DESTEKLENMESİ HAKKINDA YÖNETMELİK"in amacı, ondan az çalışanı bulunan tehlikeli ve çok tehlikeli sınıftaki işyerlerine 

QueryResponse(matches=[{'id': 'isg_274',
 'metadata': {'chunk_id': 274},
 'score': 0.733704627,
 'values': []}, {'id': 'isg_217',
 'metadata': {'chunk_id': 217},
 'score': 0.700519621,
 'values': []}, {'id': 'isg_161',
 'metadata': {'chunk_id': 161},
 'score': 0.665289402,
 'values': []}, {'id': 'isg_8', 'metadata': {'chunk_id': 8}, 'score': 0.651706219, 'values': []}, {'id': 'isg_158',
 'metadata': {'chunk_id': 158},
 'score': 0.650195062,
 'values': []}], namespace='', usage={'read_units': 1}, _response_info={'raw_headers': {'date': 'Wed, 14 Jan 2026 20:03:19 GMT', 'content-type': 'application/json', 'content-length': '446', 'connection': 'keep-alive', 'x-pinecone-max-indexed-lsn': '5', 'x-pinecone-request-latency-ms': '48', 'x-pinecone-request-id': '9046159691532446320', 'x-envoy-upstream-service-time': '48', 'x-pinecone-response-duration-ms': '50', 'grpc-status': '0', 'server': 'envoy'}})

In [53]:
query_gemini2 = "Bir inşaat şantiyesinde yüksekten düşme riski tespit edilmiştir. Mevzuata göre bu riski önlemek için alınması gereken teknik ve idari önlemler neler olmalıdır?"
run_query(query_gemini2, use_ai=True, ai_provider="gemini", use_reasoning=True, max_length="long")
print("-" * 50)

🧠 Deep Analysis with Gemini 1.5 Pro:

💭 Used Gemini 1.5 Flash (with reasoning mode)

Sağlanan mevzuat metinlerine göre, bir inşaat şantiyesinde yüksekten düşme riskini önlemek için alınması gereken teknik ve idari önlemler aşağıda belirtilmiştir:

Mevzuat metinlerinde yüksekten düşme riskine özel teknik önlemler (korkuluk, güvenlik ağı vb.) açıkça belirtilmemekle birlikte, genel risk yönetimi ve idari yükümlülükler çerçevesinde aşağıdaki adımlar atılmalıdır:

*   **Risklerin Belirlenmesi ve En Aza İndirilmesi:**
    *   İşyerindeki risklerin en aza indirilmesi için uygun tedbirler alınmalıdır. Bu, yüksekten düşme riskinin varlığı tespit edildiğinde, bu riski ortadan kaldırmaya veya minimize etmeye yönelik genel bir yükümlülüktür.
        *   *Referans:* "b) Bu işlerle ilgili risklerin en aza indirilmesi için uygun tedbirler alınır"
*   **Detaylı Risk Değerlendirmesi ve Analizi:**
    *   Yüksekten düşme tehlikesi ayrı ayrı dikkate alınarak, bu tehlikeden kaynaklanabilecek risklerin han

In [54]:
query_gemini4 = "İş sağlığı ve güvenliği kurullarının kurulması neden zorunludur ve bu kurulların aldığı kararların işveren üzerindeki bağlayıcılığı nedir?"
run_query(query_gemini4, use_ai=True, ai_provider="gemini", use_reasoning=False, max_length="long")


🤖 Enhanced Response with Gemini 1.5 Flash:

🤖 Enhanced Response with Gemini 1.5 Flash:

Sağlanan mevzuat metinleri incelendiğinde, iş sağlığı ve güvenliği kurullarının kurulmasının neden zorunlu olduğuna ve bu kurulların aldığı kararların işveren üzerindeki bağlayıcılığına dair doğrudan ve detaylı bilgi bulunmamaktadır. Ancak, metinlerde kurulun çalışanlar açısından bir işlevi belirtilmiştir:

*   **İş Sağlığı ve Güvenliği Kurullarının Kurulmasının Zorunluluğu:**
    *   Mevzuat metinlerinde, iş sağlığı ve güvenliği kurullarının hangi koşullar altında veya neden zorunlu olarak kurulması gerektiğine dair açık bir hüküm veya gerekçe yer almamaktadır. Metinlerde "kurulun bulunmadığı işyerlerinde" ifadesi geçmekle birlikte, kurulların kurulma zorunluluğunun kapsamı veya kriterleri (örneğin çalışan sayısı veya tehlike sınıfı gibi) belirtilmemiştir.

*   **Kurul Kararlarının İşveren Üzerindeki Bağlayıcılığı:**
    *   Sağlanan mevzuat metinlerinde, iş sağlığı ve güvenliği kurullarının aldığı

QueryResponse(matches=[{'id': 'isg_211',
 'metadata': {'chunk_id': 211},
 'score': 0.734601915,
 'values': []}, {'id': 'isg_212',
 'metadata': {'chunk_id': 212},
 'score': 0.727824211,
 'values': []}, {'id': 'isg_217',
 'metadata': {'chunk_id': 217},
 'score': 0.707886696,
 'values': []}, {'id': 'isg_276',
 'metadata': {'chunk_id': 276},
 'score': 0.699440897,
 'values': []}, {'id': 'isg_107',
 'metadata': {'chunk_id': 107},
 'score': 0.693913877,
 'values': []}], namespace='', usage={'read_units': 1}, _response_info={'raw_headers': {'date': 'Wed, 14 Jan 2026 20:06:23 GMT', 'content-type': 'application/json', 'content-length': '450', 'connection': 'keep-alive', 'x-pinecone-max-indexed-lsn': '5', 'x-pinecone-request-latency-ms': '47', 'x-pinecone-request-id': '1314068375839351700', 'x-envoy-upstream-service-time': '47', 'x-pinecone-response-duration-ms': '48', 'grpc-status': '0', 'server': 'envoy'}})

In [55]:
query_gemini5 = "Kimyasal maddelerle çalışılan bir işyerinde, çalışanların maruziyetini önlemek için alınması gereken öncelikli tedbirler (ikame, teknik önlemler vb.) sırasıyla nelerdir?"
run_query(query_gemini5, use_ai=True, ai_provider="gemini", use_reasoning=True, max_length="long")

🧠 Deep Analysis with Gemini 1.5 Pro:

💭 Used Gemini 1.5 Flash (with reasoning mode)

Kimyasal maddelerle çalışılan bir işyerinde, çalışanların maruziyetini önlemek için alınması gereken öncelikli tedbirler, sağlanan mevzuat metinlerine göre sırasıyla aşağıdaki gibidir:

*   **Tehlikeli Olanı, Tehlikesiz veya Daha Az Tehlikeli Olanla Değiştirmek (İkame):**
    *   Bu, kimyasal maruziyetin önlenmesinde en öncelikli ve etkili adımdır. Mevzuat metninde açıkça belirtildiği üzere, "Tehlikeli olanı, tehlikesiz veya daha az tehlikeli olanla değiştirmek (ikame)" esastır. Bu ilke, tehlikeli bir kimyasal maddenin yerine daha az riskli bir alternatifin bulunmasını ve kullanılmasını gerektirir.
    *   **Pratik Örnek:** Yüksek toksisiteye sahip bir solvent yerine, aynı işlevi gören ancak daha düşük toksisiteye sahip veya su bazlı bir alternatifin kullanılması. Örneğin, organik solvent bazlı temizleyiciler yerine biyolojik veya su bazlı temizleyicilere geçiş yapmak.

*   **Toplu Korunma Tedbirlerine

QueryResponse(matches=[{'id': 'isg_308',
 'metadata': {'chunk_id': 308},
 'score': 0.694906235,
 'values': []}, {'id': 'isg_231',
 'metadata': {'chunk_id': 231},
 'score': 0.688191414,
 'values': []}, {'id': 'isg_299',
 'metadata': {'chunk_id': 299},
 'score': 0.674422324,
 'values': []}, {'id': 'isg_311',
 'metadata': {'chunk_id': 311},
 'score': 0.671709955,
 'values': []}, {'id': 'isg_102',
 'metadata': {'chunk_id': 102},
 'score': 0.670761168,
 'values': []}], namespace='', usage={'read_units': 1}, _response_info={'raw_headers': {'date': 'Wed, 14 Jan 2026 20:07:25 GMT', 'content-type': 'application/json', 'content-length': '450', 'connection': 'keep-alive', 'x-pinecone-max-indexed-lsn': '5', 'x-pinecone-request-latency-ms': '6', 'x-pinecone-request-id': '1504615211750440096', 'x-envoy-upstream-service-time': '6', 'x-pinecone-response-duration-ms': '8', 'grpc-status': '0', 'server': 'envoy'}})

In [57]:

query_compare = "Kimyasal maddelerle çalışılan bir işyerinde, çalışanların maruziyetini önlemek için alınması gereken öncelikli tedbirler (ikame, teknik önlemler vb.) sırasıyla nelerdir?"

run_query(query_compare, use_ai=True, ai_provider="openai", max_length="long")



🤖 Enhanced Response with GPT-4 Turbo:

Kimyasal maddelerle çalışılan bir işyerinde, çalışanların maruziyetini önlemek için alınması gereken öncelikli tedbirler aşağıdaki gibidir:

- **Tehlikeli olanı, tehlikesiz veya daha az tehlikeli olanla değiştirmek (ikame)**: İşveren, kimyasal maddeleri daha az tehlikeli alternatiflerle değiştirmelidir. Örneğin, toksik bir kimyasal yerine daha az zararlı bir madde kullanmak.

- **Teknik önlemler almak**: İşyerinde kimyasal maddelerin kullanımı sırasında gerekli teknik önlemler alınmalıdır. Bu, havalandırma sistemlerinin kurulması veya kimyasal maddelerin otomatik sistemlerle taşınması gibi uygulamaları içerebilir.

- **Toplu korunma tedbirlerine öncelik vermek**: Çalışanların bireysel korunma tedbirlerinden önce toplu korunma tedbirleri uygulanmalıdır. Örneğin, kimyasal maddelerin depolandığı alanların güvenli bir şekilde tasarlanması ve bu alanların erişimine sınırlamalar getirilmesi.

- **Çalışanlara uygun talimatlar vermek**: İşveren, çalışanla

QueryResponse(matches=[{'id': 'isg_308',
 'metadata': {'chunk_id': 308},
 'score': 0.694906235,
 'values': []}, {'id': 'isg_231',
 'metadata': {'chunk_id': 231},
 'score': 0.688191414,
 'values': []}, {'id': 'isg_299',
 'metadata': {'chunk_id': 299},
 'score': 0.674422324,
 'values': []}, {'id': 'isg_311',
 'metadata': {'chunk_id': 311},
 'score': 0.671709955,
 'values': []}, {'id': 'isg_102',
 'metadata': {'chunk_id': 102},
 'score': 0.670761168,
 'values': []}], namespace='', usage={'read_units': 1}, _response_info={'raw_headers': {'date': 'Wed, 14 Jan 2026 20:14:49 GMT', 'content-type': 'application/json', 'content-length': '450', 'connection': 'keep-alive', 'x-pinecone-max-indexed-lsn': '5', 'x-pinecone-request-latency-ms': '55', 'x-pinecone-request-id': '4112780726222353994', 'x-envoy-upstream-service-time': '56', 'x-pinecone-response-duration-ms': '57', 'grpc-status': '0', 'server': 'envoy'}})